In this example, we utilize the 10x Genomics Multiome data provided by the NeurIPS 2021 Multimodal Single-Cell Data Integration competition. This data can be downloaded from NCBI GEO with the accession number [GSE194122](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE194122).

To run this example, users need to ensure that pandas, numpy, scikit-learn, and datatable are installed in the runtime environment. Datatable, in particular, supports rapid reading and writing of large-scale files.

Please download the h5ad file before running the code below.
```bash
wget -O - "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE194nnn/GSE194122/suppl/GSE194122%5Fopenproblems%5Fneurips2021%5Fmultiome%5FBMMC%5Fprocessed.h5ad.gz" | pigz -d > GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad
```

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc

import datatable as dt

from sklearn.preprocessing import OneHotEncoder


/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read_h5ad('GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad')

### Solely use the transcriptomic data

In [3]:
genes=adata.var_names[adata.var['feature_types']=='GEX']
adata_gex=adata[:,genes].copy()
adata_gex.X = adata_gex.layers['counts'].copy()

#### Select 2000 highly variable genes (HVGs)

In [4]:
sc.pp.normalize_total(adata_gex, target_sum=1e4)
sc.pp.log1p(adata_gex)
sc.pp.highly_variable_genes(adata_gex, n_top_genes=2000, batch_key='batch', subset=True)

/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:489: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hvg = pd.concat([hvg, missing_hvg], ignore_index=True)
/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default o

#### Save counts, cell names, and HVG names to local files

In [5]:
dt.Frame(pd.DataFrame(adata_gex.layers['counts'].toarray(), columns=adata_gex.var_names, dtype=np.float64)).to_csv('multiome_neurips21_counts.txt.gz')


adata_gex.obs_names.to_frame().to_csv('multiome_neurips21_cell.csv.gz', index=None)
adata_gex.var_names.to_frame().to_csv('multiome_neurips21_gene.csv.gz', index=None)

#### Save undesired factor values to local file

In [6]:
enc = OneHotEncoder(sparse_output=False).fit(adata_gex.obs['batch'].to_numpy().reshape(-1,1))
pd.DataFrame(enc.transform(adata_gex.obs['batch'].to_numpy().reshape(-1,1)), columns=enc.categories_).to_csv('multiome_neurips21_uwv.txt.gz', index=False)

Different versions of Scikit-learn's OneHotEncoder may have different parameters. Some versions use `sparse`, while others use `sparse_output`. Users need to adjust the parameters according to their own environment.